In [18]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error
import pickle

## Train Test Split

In [19]:
import os
# Office
os.chdir('C:\\Users\\prudi\\Documents\\GitHub\\Code\\NITDATA\\Algorithms\\codesnippetsforbaggingandboostingalgorithms')
data = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data.drop('Loan_Status',axis=1),data['Loan_Status'],test_size=0.30,random_state=42)
X_train.shape, X_test.shape

((429, 12), (185, 12))

In [21]:
## Using the above Pickle Files

In [22]:
X_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
553,LP002788,Male,Yes,0,Not Graduate,No,2454,2333.0,181.0,360.0,0.0,Urban
601,LP002950,Male,Yes,0,Not Graduate,NaN,2894,2792.0,155.0,360.0,1.0,Rural
261,LP001868,Male,No,0,Graduate,No,2060,2209.0,134.0,360.0,1.0,Semiurban
496,LP002587,Male,Yes,0,Not Graduate,No,2600,1700.0,107.0,360.0,1.0,Rural
529,LP002716,Male,No,0,Not Graduate,No,6783,0.0,130.0,360.0,1.0,Semiurban


In [23]:
X_train=X_train.reset_index(drop=True)
X_train.loc[X_train['Gender'].isnull(),'Gender']='Trasgender'
X_train.loc[X_train['Dependents'].isnull(),'Dependents']='0'
X_train.loc[X_train['Married'].isnull(),'Married']='No'
X_train.loc[X_train['Self_Employed'].isnull(),'Self_Employed']='No'
X_train['LoanAmount'] = X_train['LoanAmount'].fillna(146.8671497584541)
X_train.loc[X_train['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=0
X_train.loc[X_train['Credit_History'].isnull(),'Credit_History']=0


X_train=X_train.drop(['Loan_ID'],axis=1)

X_train['Credit_History']=X_train['Credit_History'].astype(str)

X_train=X_train.reset_index(drop=True)

In [24]:
X_train['Loan_Amount_Term'].mean()

330.013986013986

In [25]:
print(X_train.shape)

(429, 11)


In [26]:
#loading a MINMAX Scaler from a file called minmax_pickle.pkl
## Creating a Instance for the minmax Pickle File
MinMaxpickle_in = open("minmax_pickle.pkl","rb")
        
## Loading the Pickle
MinmaxScaler_dict = pickle.load(MinMaxpickle_in)
        
## Applying the pickle file
X_train['ApplicantIncome']=pd.DataFrame(MinmaxScaler_dict['ApplicantIncome'].transform(X_train[['ApplicantIncome']]))

In [27]:
Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        
## Loading the Pickle
Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
## Applying the pickle file
X_train['Education']=pd.DataFrame(Labelencoder_dict['Education'].transform(X_train[['Education']]))
X_train['Property_Area']=pd.DataFrame(Labelencoder_dict['Property_Area'].fit_transform(X_train[['Property_Area']]))
X_train['Dependents']=pd.DataFrame(Labelencoder_dict['Dependents'].fit_transform(X_train[['Dependents']]))
X_train['Credit_History']=pd.DataFrame(Labelencoder_dict['Credit_History'].fit_transform(X_train[['Credit_History']]))

In [28]:
X_train['Credit_History'].unique()

array([0, 1], dtype=int64)

In [29]:
Onehotpickle_in = open("binarizer_pickle.pkl","rb")
        
## Loading the Pickle
Onehot_dict = pickle.load(Onehotpickle_in)
Onehot_dict   
## Applying the pickle file
d1=pd.DataFrame(Onehot_dict['Self_Employed'].transform(X_train[['Self_Employed']]))
d1.columns=['Self_Employed_0']
#d1.head(1)

In [30]:
## Applying the pickle file
d2=pd.DataFrame(Onehot_dict['Married'].transform(X_train[['Married']]))
d2.columns=['Marrioed_0']
d2.head(1)

,Marrioed_0
0,1


In [31]:
## Applying the pickle file
d3=pd.DataFrame(Onehot_dict['Gender'].transform(X_train[['Gender']]))
d3.columns=['a','b','c']
d3.head(1)

,a,b,c
0,0,1,0


In [32]:
d=pd.concat([d1,d2,d3],axis=1)
col=['Self_Employed_0','Married_0','Gender_0','Gender_3','Gender_2']
d.columns=col
X_train.drop(['Self_Employed','Married','Gender'],axis=1,inplace=True)
X_train=pd.concat([X_train,d],axis=1)

In [33]:
X_train.head()

,Dependents,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Self_Employed_0,Married_0,Gender_0,Gender_3,Gender_2
0,0,1,0.175717,2333.0,181.0,360.0,0,2,0,1,0,1,0
1,0,1,0.209274,2792.0,155.0,360.0,1,0,0,1,0,1,0
2,0,0,0.145668,2209.0,134.0,360.0,1,1,0,0,0,1,0
3,0,1,0.186852,1700.0,107.0,360.0,1,0,0,1,0,1,0
4,0,1,0.505872,0.0,130.0,360.0,1,1,0,0,0,1,0
